In [1]:
#!/usr/bin/python
#
# (c) 2017 Gergely Katona <gergely.katona@gu.se>
#
# Reference: Garcia-Bonete MJ, Katona G. Bayesian machine learning improves single-wavelength anomalous diffraction phasing. Acta Cryst A. 2019;75:851-60.
#

import sys
import numpy
import scipy
import os
import shutil
import scipy.stats
import scipy.integrate
import pymc3 as pm
import scipy as sp
from theano import tensor as T
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import iotbx
from cctbx.array_family import flex
from cctbx import crystal
from cctbx import uctbx
from cctbx import sgtbx
from cctbx import miller
from iotbx import reflection_file_reader, mtz


In [4]:
def initializecrystal():
    unit_cell=( 79.07,    79.07,    36.98,  90.000,  90.000,  90.000 )       
    uc = uctbx.unit_cell(unit_cell)
    wavelength = 1.54980
    xtal_symm = crystal.symmetry(unit_cell=unit_cell, space_group_symbol="P 43 21 2")

    ms = miller.build_set(crystal_symmetry=xtal_symm,anomalous_flag=True, d_min=1.61)
    msnam = miller.build_set(crystal_symmetry=xtal_symm,anomalous_flag=False, d_min=1.61)

    mscent=list(ms.select_centric().indices())
    msacent=list(ms.select_acentric().indices())
    msnamacent=list(msnam.select_acentric().indices())
    dstar=msnam.d_star_sq()
    msnamacent_dstar=list(dstar.select_acentric().data())
    msnamcent=list(msnam.select_centric().indices())
    return ms,msnam,mscent,msacent,msnamacent,msnamcent,msnamacent_dstar

ms,msnam,mscent,msacent,msnamacent,msnamcent,msnamacent_dstar=initializecrystal()

In [ ]:
#Inverse treatment
def readset(roots):
    trows=[]

    for root in roots:
        drows=[]
        ms,msnam,mscent,msacent,msnamacent,msnamcent,msnamacent_dstar=initializecrystal()
        
        data1=iotbx.xds.read_ascii.reader(open(root+"_run1.hkl"))
        data2=iotbx.xds.read_ascii.reader(open(root+"_run2.hkl"))

        inten1=data1.as_miller_array(merge_equivalents=False).as_anomalous_array()
        batch1=data1.batch_as_miller_array().as_anomalous_array()

        inten2=data2.as_miller_array(merge_equivalents=False).as_anomalous_array()
        batch2=data2.batch_as_miller_array().as_anomalous_array()

        mapinten1=inten1.map_to_asu()
        #print list(mapinten1)
        mapbatch1=batch1.map_to_asu()

        mapinten2=inten2.map_to_asu()
        mapbatch2=batch2.map_to_asu()

        #data.zd.as_numpy_array()

        liinten1=list(mapinten1)
        libatch1=list(mapbatch1)

        liinten2=list(mapinten2)
        libatch2=list(mapbatch2)

        prezipref1=zip(liinten1,libatch1)
        prezipref2=zip(liinten2,libatch2)


        for idx,odd in enumerate(prezipref1):
            drow={'hkl':odd[0][0],
                          'I':odd[0][1],
                          'SIGI':odd[0][2],
                          'batch':odd[1][1],
                          'idx': idx,
                          'face': 'h',
                          'root':root,
                          'treated': False
                         }
            drows.append(drow)
        print ("File1 read in")

        for idx,even in enumerate(prezipref2):
            drow={'hkl':even[0][0],
                          'I':even[0][1],
                          'SIGI':even[0][2],
                          'batch':even[1][1],
                          'idx': idx,
                          'face': 't',
                          'root':root,
                          'treated': False
                         }
            drows.append(drow)
            
        print ("File2 read in")
        df = pd.DataFrame(drows)

# Paired one way

        i=0
        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==etind)&(df.face=='h')&(df.treated==False)&(df.SIGI>0)].index.tolist():
                try:
                    tempf=abs(df[(df.hkl==invetind)&(df.face=='t')&(df.treated==False)&(df.SIGI>0)].batch-df.iloc[item1].batch)
                    item2=tempf.idxmin
                    if abs(df.iloc[item1].batch-df.iloc[item2].batch)<10:
                        trow={
                          'hkl':df.iloc[item1].hkl,
                          'Iplus':df.iloc[item1].I,
                          'SIGIplus':df.iloc[item1].SIGI,
                          'batchplus':df.iloc[item1].batch,
                          'Iminus':df.iloc[item2].I,
                          'SIGIminus':df.iloc[item2].SIGI,
                          'batchminus':df.iloc[item2].batch,
                          'root':df.iloc[item1].root,
                          'paired':'pair',
                         }
                        trows.append(trow)
                        df.set_value(item1,'treated',True)
                        df.set_value(item2,'treated',True)
                        i=i+1
                except:
                    pass
                
        print ("Phase 1 completed")

# Paired reverse

        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==invetind)&(df.face=='h')&(df.treated==False)&(df.SIGI>0)].index.tolist():
                try:
                    tempf=abs(df[(df.hkl==etind)&(df.face=='t')&(df.treated==False)&(df.SIGI>0)].batch-df.iloc[item1].batch)
                    item2=tempf.idxmin

                    if abs(df.iloc[item1].batch-df.iloc[item2].batch)<10:
                        trow={
                          'hkl':df.iloc[item2].hkl,
                          'Iplus':df.iloc[item2].I,
                          'SIGIplus':df.iloc[item2].SIGI,
                          'batchplus':df.iloc[item2].batch,
                          'Iminus':df.iloc[item1].I,
                          'SIGIminus':df.iloc[item1].SIGI,
                          'batchminus':df.iloc[item1].batch,
                          'root':df.iloc[item1].root,
                          'paired':'pair',
                         }
                        trows.append(trow)
                        df.set_value(item1,'treated',True)
                        df.set_value(item2,'treated',True)
                        i=i+1
                except:
                    pass

        print ("Phase 2 completed")
                
# Solos +

        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==etind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                trow={
                      'hkl':df.iloc[item1].hkl,
                      'Iplus':df.iloc[item1].I,
                      'SIGIplus':df.iloc[item1].SIGI,
                      'batchplus':df.iloc[item1].batch,
                      'root':df.iloc[item1].root,
                      'paired':'plus',
                         }
                trows.append(trow)
                df.set_value(item1,'treated',True)

# Solos - 

        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==invetind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                trow={
                      'hkl':df.iloc[item1].hkl,
                      'Iminus':df.iloc[item1].I,
                      'SIGIminus':df.iloc[item1].SIGI,
                      'batchminus':df.iloc[item1].batch,
                      'root':df.iloc[item1].root,
                      'paired':'minus',
                         }
                trows.append(trow)
                df.set_value(item1,'treated',True)

                        
# Centric - 

        for etind in mscent:
            for item1 in df[(df.hkl==etind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                trow={
                          'hkl':df.iloc[item1].hkl,
                          'Icent':df.iloc[item1].I,
                          'SIGIcent':df.iloc[item1].SIGI,
                          'batchcent':df.iloc[item1].batch,
                          'root':df.iloc[item1].root,
                          'paired':'cent',
                         }
                trows.append(trow)
                df.set_value(item1,'treated',True)

                        
        print (root+" completed! Found matching reflections:"+str(i)+".")
    
    tf = pd.DataFrame(trows)
    return tf

GHz400=['inverse1','inverse2','inverse3','inverse4']
GHz400.sort()
tf =readset(GHz400)

File1 read in
File2 read in


In [44]:
# Continuous treatment for Bijvoet pairs!
exptype='CONT'
def readset(roots):
    
    trows=[]

    for root in roots:
        drows=[]

        ms,msnam,mscent,msacent,msnamacent,msnamcent,msnamacent_dstar=initializecrystal()
        
        data1=iotbx.xds.read_ascii.reader(open(root+".hkl"))

        inten1=data1.as_miller_array(merge_equivalents=False).as_anomalous_array()
        batch1=data1.batch_as_miller_array().as_anomalous_array()



        mapinten1=inten1.map_to_asu()
        mapbatch1=batch1.map_to_asu()



        liinten1=list(mapinten1)
        libatch1=list(mapbatch1)


        prezipref1=zip(liinten1,libatch1)


        for idx,odd in enumerate(prezipref1):
            drow={'hkl':odd[0][0],
                          'I':odd[0][1],
                          'SIGI':odd[0][2],
                          'batch':odd[1][1],
                          'idx': idx,
                          'root':root,
                          'treated': False
                         }
            drows.append(drow)
        print ("File1 read in")


        df = pd.DataFrame(drows)

# Paired one way

        i=0
        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==etind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                try:
                    tempf=abs(df[(df.hkl==invetind)&(df.root==df.iloc[item1].root)&(df.treated==False)&(df.SIGI>0)].batch-df.iloc[item1].batch)
                    item2=tempf.idxmin
                    if abs(df.iloc[item1].batch-df.iloc[item2].batch)<100:
                        trow={
                          'hkl':etind,
                          'Iplus':df.iloc[item1].I,
                          'SIGIplus':df.iloc[item1].SIGI,
                          'batchplus':df.iloc[item1].batch,
                          'Iminus':df.iloc[item2].I,
                          'SIGIminus':df.iloc[item2].SIGI,
                          'batchminus':df.iloc[item2].batch,
                          'root':df.iloc[item1].root,
                          'paired':'pair',
                         }
                        trows.append(trow)
                        df.set_value(item1,'treated',True)
                        df.set_value(item2,'treated',True)
                        i=i+1
                except:
                    pass
                
        print ("Phase 1 completed")

# Paired reverse

        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==invetind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                try:
                    tempf=abs(df[(df.hkl==etind)&(df.root==df.iloc[item1].root)&(df.treated==False)&(df.SIGI>0)].batch-df.iloc[item1].batch)
                    item2=tempf.idxmin

                    if abs(df.iloc[item1].batch-df.iloc[item2].batch)<100:
                        trow={
                          'hkl':etind,
                          'Iplus':df.iloc[item2].I,
                          'SIGIplus':df.iloc[item2].SIGI,
                          'batchplus':df.iloc[item2].batch,
                          'Iminus':df.iloc[item1].I,
                          'SIGIminus':df.iloc[item1].SIGI,
                          'batchminus':df.iloc[item1].batch,
                          'root':df.iloc[item1].root,
                          'paired':'pair',
                         }
                        trows.append(trow)
                        df.set_value(item1,'treated',True)
                        df.set_value(item2,'treated',True)
                        i=i+1
                except:
                    pass

        print ("Phase 2 completed")
                
# Solos +

        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==etind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                trow={
                      'hkl':etind,
                      'Iplus':df.iloc[item1].I,
                      'SIGIplus':df.iloc[item1].SIGI,
                      'batchplus':df.iloc[item1].batch,
                      'root':df.iloc[item1].root,
                      'paired':'plus',
                         }
                trows.append(trow)
                df.set_value(item1,'treated',True)

# Solos - 

        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==invetind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                trow={
                      'hkl':etind,
                      'Iminus':df.iloc[item1].I,
                      'SIGIminus':df.iloc[item1].SIGI,
                      'batchminus':df.iloc[item1].batch,
                      'root':df.iloc[item1].root,
                      'paired':'minus',
                         }
                trows.append(trow)
                df.set_value(item1,'treated',True)

# Centric - 

        for etind in msnamcent:
            for item1 in df[(df.hkl==etind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                trow={
                          'hkl':etind,
                          'Icent':df.iloc[item1].I,
                          'SIGIcent':df.iloc[item1].SIGI,
                          'batchcent':df.iloc[item1].batch,
                          'root':df.iloc[item1].root,
                          'paired':'cent',
                         }
                trows.append(trow)
                df.set_value(item1,'treated',True)

        print (root+" completed! Found matching reflections:"+str(i)+".")
    
    tf = pd.DataFrame(trows)
    return (tf)

datasets=['cont2','cont3','cont4','cont5','cont6']
datasets.sort()
tf =readset(datasets)

File1 read in
Phase 1 completed
Phase 2 completed
File1 read in
Phase 1 completed
Phase 2 completed
File1 read in
Phase 1 completed
Phase 2 completed
File1 read in
Phase 1 completed
Phase 2 completed
File1 read in
Phase 1 completed
Phase 2 completed
cont6 completed! Found matching reflections:67627.


In [37]:
# Continuous treatment, but pairing Friedels!!!
exptype='CONT'
def readset(roots):
    
    trows=[]

    for root in roots:
        drows=[]

        ms,msnam,mscent,msacent,msnamacent,msnamcent,msnamacent_dstar=initializecrystal()
        
        data1=iotbx.xds.read_ascii.reader(open(root+".hkl"))

        inten1=data1.as_miller_array(merge_equivalents=False).as_anomalous_array()
        batch1=data1.batch_as_miller_array().as_anomalous_array()

        mapinten1=inten1.map_to_asu()
        mapbatch1=batch1.map_to_asu()

        liinten1=list(mapinten1)
        libatch1=list(mapbatch1)

        prezipref1=zip(liinten1,libatch1)

        for idx,odd in enumerate(prezipref1):
            drow={'hkl':odd[0][0],
                          'I':odd[0][1],
                          'SIGI':odd[0][2],
                          'batch':odd[1][1],
                          'idx': idx,
                          'root':root,
#                          'cent': odd[0][0] in mscent,
                          'treated': False
                         }
            drows.append(drow)
        print ("File1 read in")

        df = pd.DataFrame(drows)

# Paired one way

        i=0
        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==etind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                try:
                    tempf=abs(abs(df[(df.hkl==invetind)&(df.root==df.iloc[item1].root)&(df.treated==False)&(df.SIGI>0)].batch-df.iloc[item1].batch)-1800)
                    item2=tempf.idxmin
                    if abs(abs(df.iloc[item1].batch-df.iloc[item2].batch)-1800)<10:
                        trow={
                          'hkl':etind,
                          'Iplus':df.iloc[item1].I,
                          'SIGIplus':df.iloc[item1].SIGI,
                          'batchplus':df.iloc[item1].batch,
                          'Iminus':df.iloc[item2].I,
                          'SIGIminus':df.iloc[item2].SIGI,
                          'batchminus':df.iloc[item2].batch,
                          'root':df.iloc[item1].root,
                          'paired':'pair',
                         }
                        trows.append(trow)
                        df.set_value(item1,'treated',True)
                        df.set_value(item2,'treated',True)
                        i=i+1
                except:
                    pass
                
        print ("Phase 1 completed")

# Paired reverse

        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==invetind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                try:
                    tempf=abs(abs(df[(df.hkl==etind)&(df.root==df.iloc[item1].root)&(df.treated==False)&(df.SIGI>0)].batch-df.iloc[item1].batch)-1800)
                    item2=tempf.idxmin
                    if abs(abs(df.iloc[item1].batch-df.iloc[item2].batch)-1800)<10:
                        trow={
                          'hkl':etind,
                          'Iplus':df.iloc[item2].I,
                          'SIGIplus':df.iloc[item2].SIGI,
                          'batchplus':df.iloc[item2].batch,
                          'Iminus':df.iloc[item1].I,
                          'SIGIminus':df.iloc[item1].SIGI,
                          'batchminus':df.iloc[item1].batch,
                          'root':df.iloc[item1].root,
                          'paired':'pair',
                         }
                        trows.append(trow)
                        df.set_value(item1,'treated',True)
                        df.set_value(item2,'treated',True)
                        i=i+1
                except:
                    pass

        print ("Phase 2 completed")
                
# Solos +

        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==etind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                trow={
                      'hkl':etind,
                      'Iplus':df.iloc[item1].I,
                      'SIGIplus':df.iloc[item1].SIGI,
                      'batchplus':df.iloc[item1].batch,
                      'root':df.iloc[item1].root,
                      'paired':'plus',
                         }
                trows.append(trow)
                df.set_value(item1,'treated',True)

# Solos - 

        for etind in msnamacent:
            invetind=tuple([-1*x for x in etind])
            for item1 in df[(df.hkl==invetind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                trow={
                      'hkl':etind,
                      'Iminus':df.iloc[item1].I,
                      'SIGIminus':df.iloc[item1].SIGI,
                      'batchminus':df.iloc[item1].batch,
                      'root':df.iloc[item1].root,
                      'paired':'minus',
                         }
                trows.append(trow)
                df.set_value(item1,'treated',True)

# Centric - 

        for etind in msnamcent:
            for item1 in df[(df.hkl==etind)&(df.treated==False)&(df.SIGI>0)].index.tolist():
                trow={
                          'hkl':etind,
                          'Icent':df.iloc[item1].I,
                          'SIGIcent':df.iloc[item1].SIGI,
                          'batchcent':df.iloc[item1].batch,
                          'root':df.iloc[item1].root,
                          'paired':'cent',
                         }
                trows.append(trow)
                df.set_value(item1,'treated',True)

        print (root+" completed! Found matching reflections:"+str(i)+".")
    
    tf = pd.DataFrame(trows)
    return (tf)

datasets=['cont2','cont3','cont4','cont5','cont6']
datasets.sort()
tf =readset(datasets)

File1 read in


/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:69: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:70: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Phase 1 completed
Phase 2 completed


/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:120: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:136: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:151: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


cont2 completed! Found matching reflections:132943.
File1 read in
Phase 1 completed
Phase 2 completed
cont3 completed! Found matching reflections:124439.
File1 read in
Phase 1 completed
Phase 2 completed
cont4 completed! Found matching reflections:113281.
File1 read in
Phase 1 completed
Phase 2 completed
cont5 completed! Found matching reflections:109072.
File1 read in
Phase 1 completed
Phase 2 completed
cont6 completed! Found matching reflections:113161.


In [79]:
def multidiff(refl):           
    with pm.Model() as model:
        quality=0
        F = pm.Uniform('F',0,1e8,shape=2)
        sigma = pm.Lognormal('sigma', 0, 1,shape=2)

        Fdel = pm.Deterministic('Fdel',F[0]-F[1])
        Fmean = pm.Deterministic('Fmean',(F[0]+F[1])/2.0)
        Effect = pm.Deterministic('Effect',(F[0]**2-F[1]**2)/pm.math.sqrt(sigma[0]**2+sigma[1]**2))

        C_t = pm.LKJCorr('C_t', 1, 2) 
        C = pm.Deterministic('C', T.fill_diagonal(C_t[np.zeros((2, 2), dtype=np.int64)], 1.))
    
        sigma_diag = pm.Deterministic('sigma_mat', T.nlinalg.diag(sigma))
        cov = pm.Deterministic('cov', T.nlinalg.matrix_dot(sigma_diag, C, sigma_diag))
        obsMV = pm.MvNormal('MV',pm.math.sqr(F),cov=cov, observed=tf[(tf.hkl==(refl))&(tf.paired=='pair')][['Iplus','Iminus']])


    with model:
        step = pm.Metropolis()
        trace = pm.sample(10000,tune=40000,step=step, progressbar=True)

    return trace

def uni(refl, rtype):
    itype={}
    itype['cent']='Icent'
    itype['plus']='Iplus'
    itype['minus']='Iminus'
    
    with pm.Model() as model:
        F = pm.Uniform('F',0,1e8)    
        sigma = pm.Lognormal('sigma', 0, 1)
        Fmean = pm.Deterministic('Fmean',F)

        obs = pm.Normal('obs',pm.math.sqr(F),sd=sigma, observed=tf[(tf.hkl==(refl))&(tf.paired==rtype)][[itype[rtype]]])

    with model:
        step = pm.Metropolis()
        trace_ = pm.sample(50000, step, progressbar=True)
        trace = trace_[40000:]
        
    return trace
 
def unidiff(refl):
    with pm.Model() as model:
        F = pm.Uniform('F',0,1e8,shape=2)  
        sigma = pm.Lognormal('sigma', 0, 1)
        Fmean = pm.Deterministic('Fmean',F)
        Fdel = pm.Deterministic('Fdel',F[0]-F[1])
        Effect = pm.Deterministic('Effect',(F[0]**2-F[1]**2)/sigma)
        
        obsplus = pm.Normal('Plus',pm.math.sqr(F[0]),sd=sigma, observed=tf[(tf.hkl==(refl))][['Iplus']])
        obsminus = pm.Normal('Minus',pm.math.sqr(F[1]),sd=sigma, observed=tf[(tf.hkl==(refl))][['Iminus']])

    with model:
        step = pm.Metropolis()
        trace_ = pm.sample(50000, step, progressbar=True)
        trace = trace_[40000:]

    return trace

In [ ]:
    
ms,msnam,mscent,msacent,msnamacent=initializecrystal()

for refl in msnamacent:
    if tf[(tf.hkl==(refl))].shape[0]==0:
        print ("No acentric reflection!")        
    elif (tf[(tf.hkl==(refl))&(tf.paired=='pair')].shape[0]==0) & (tf[(tf.hkl==(refl))&(tf.paired=='minus')].shape[0]==0):
        trace=uni(refl,'plus')
#        f.write(str(refl[0])+','+str(refl[1])+','+str(refl[2]))
        with open('workfile.hkl', 'a') as f:
            f.write('%6d,%6d,%6d,' % (refl))
            f.write('%10.3E,%10.3E,' % (np.median(trace['Fmean']),np.std(trace['Fmean'])))
            f.write(',,,')
            f.write('1,0\n')        
        print ("Unirun")
        sys.stderr.write(str(refl)+" "+str(np.median(trace['Fmean']))+" "+"plus"+"\n")
    elif (tf[(tf.hkl==(refl))&(tf.paired=='pair')].shape[0]==0) & (tf[(tf.hkl==(refl))&(tf.paired=='plus')].shape[0]==0):
        trace=uni(refl,'minus')
        with open('workfile.hkl', 'a') as f:
            f.write('%6d,%6d,%6d,' % (refl))
            f.write('%10.3E,%10.3E,' % (np.median(trace['Fmean']),np.std(trace['Fmean'])))
            f.write(',,,')
            f.write('2,0\n')
        print ("Unirun")
        sys.stderr.write(str(refl)+" "+str(np.median(trace['Fmean']))+" "+"minus"+"\n")
    elif tf[(tf.hkl==(refl))&(tf.paired=='pair')].shape[0]>=3:
        trace=multidiff(refl)
        with open('workfile.hkl', 'a') as f:
            f.write('%6d,%6d,%6d,' % (refl))
            f.write('%10.3E,%10.3E,' % (np.median(trace['Fmean']),np.std(trace['Fmean'])))
            f.write('%10.3E,%10.3E,' % (np.median(trace['Fdel']),np.std(trace['Fdel'])))
            f.write('0,0\n')
        with open('miscfile.hkl', 'a') as f:
            f.write('%6d,%6d,%6d,' % (refl))
            f.write('%10.3E,%10.3E,' % (np.median(trace['C_t']),np.std(trace['C_t'])))
            f.write('%10.3E,%10.3E,' % (np.median(trace['Effect']),np.std(trace['Effect'])))
            f.write('%10.3E,%10.3E' % (np.median(trace['Fdel'])/np.std(trace['Fdel']),pm.autocorr(trace['Fdel'],lag=100)))
            f.write('\n')
        print ("Multirun")
        sys.stderr.write(str(refl)+" "+str(np.median(trace['Fdel']))+" "+str(np.median(trace['Effect']))+"\n")
    elif tf[(tf.hkl==(refl))&(tf.paired=='pair')].shape[0]<3:
        trace=unidiff(refl)
        with open('workfile.hkl', 'a') as f:
            f.write('%6d,%6d,%6d,' % (refl))
            f.write('%10.3E,%10.3E,' % (np.median(trace['Fmean']),np.std(trace['Fmean'])))
            f.write('%10.3E,%10.3E,' % (np.median(trace['Fdel']),np.std(trace['Fdel'])))
            f.write('0,0\n')
        with open('miscfile.hkl', 'a') as f:
            f.write('%6d,%6d,%6d,' % (refl))
            f.write(',,' )
            f.write('%10.3E,%10.3E,' % (np.median(trace['Effect']),np.std(trace['Effect'])))
            f.write('%10.3E,%10.3E' % (np.median(trace['Fdel'])/np.std(trace['Fdel']),pm.autocorr(trace['Fdel'],lag=100)))
            f.write('\n')
        print ("Unipairrun")
        sys.stderr.write(str(refl)+" "+str(np.median(trace['Fdel']))+" "+str(np.median(trace['Effect']))+"\n")

for refl in mscent:
    if tf[(tf.hkl==(refl))].shape[0]==0:
        print ("No centric reflection!")
    else:
        trace=uni(refl,'cent')
        with open('workfile.hkl', 'a') as f:
            f.write('%6d,%6d,%6d,' % (refl))
            f.write('%10.3E,%10.3E,' % (np.median(trace['Fmean']),np.std(trace['Fmean'])))
            f.write('%10.3E,%10.3E,' % (0.0,0.0))
            f.write('0,0\n')
        print ("Unirun")
        sys.stderr.write(str(refl)+" "+str(np.median(trace['Fmean']))+" "+"cent"+"\n")


